In [1]:
from ast import boolop
import torch
import torch.nn as nn

from sklearn.neural_network import MLPClassifier
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Dense, Activation, Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, RMSprop
import os
import sys
import argparse
import pickle
import numpy as np
import pandas as pd

import configparser
import argparse
import textwrap

from model_utils import *
from data_utils import _make_dataset
from gaussian import Gaussian

import dice
import gradual_construction
from bayes_cf import *
import face_knn

In [2]:
ssl = pd.read_csv("../data/datasets/ssl/ssl_numeric.csv")

In [20]:
ssl

,SSL SCORE,PREDICTOR RAT AGE AT LATEST ARREST,PREDICTOR RAT VICTIM SHOOTING INCIDENTS,PREDICTOR RAT VICTIM BATTERY OR ASSAULT,PREDICTOR RAT ARRESTS VIOLENT OFFENSES,PREDICTOR RAT NARCOTIC ARRESTS,PREDICTOR RAT TREND IN CRIMINAL ACTIVITY,PREDICTOR RAT UUW ARRESTS,RACE CODE CD
0,0,21,0,1,4,1,1.0,1,nw
1,0,29,0,9,1,0,1.0,0,w
2,0,15,1,2,1,3,1.0,0,w
3,0,17,1,1,4,0,1.6,1,nw
4,0,16,0,0,5,0,1.3,3,nw
...,...,...,...,...,...,...,...,...,...
398577,1,42,1,1,1,0,-0.1,0,nw
398578,1,37,0,0,0,0,0.3,0,nw
398579,1,76,0,0,0,1,0.1,0,nw
398580,1,47,0,0,0,1,-0.1,0,w


In [21]:
protect_idx = ssl[ssl['RACE CODE CD'] == 'nw'].index

In [22]:
config = configparser.ConfigParser()
config.read('ssl.config')
basic_config = config['Basic']
classifier_config = config['Classifier']
data_config = config['Data']
vae_config = config['Vae']
dice_config = config['DiCE']
bayes_config = config['Bayes']
grad_con_config = config['GradualConstruction']
face_config = config['FACE']

In [23]:
data = _make_dataset(
            fname=basic_config['dataset_path'],
            dag_fname = basic_config['dag_path'],
            softmax_temp = float( data_config['softmax_temp'] ),
            target_column=data_config['target_column'],
            drop_columns=eval( data_config['drop_columns'] ),
            set_na = eval( data_config['set_na'] ),
            immutable_columns = eval( data_config['immutable_columns'] ),
            nonactionable_columns = eval( data_config[ 'nonactionable_columns' ] ),
            log_transform = eval( data_config[ 'log_transform' ] ),
            dataset=basic_config['dataset'],
    )
X = data[:][0] #Get preprocessed data
y = data[:][1] #Get preprocessed labels
X_women = X[data.protect_list]
y_women = y[data.protect_list]
X_men = X[~data.protect_list]
y_men = y[~data.protect_list]

No DAG file provided/DAG path does not exist


In [24]:
X.shape

torch.Size([398582, 7])

In [25]:
use_X = pd.DataFrame(X.numpy())
def generator_small(data_dim):
    model = Sequential()
    
    model.add(Dense(128, input_dim=data_dim))
    model.add(Activation('relu'))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dense(data_dim))
    
    return model

def whiten(data, mu, sigma, dis):
    new_df = pd.DataFrame(data, columns = list(range(data.shape[1])))
    for col in new_df.columns:
        if col in []:
            new_df[col] = np.log(new_df[col] + 10e-6) 
        # if str(col) in dis or col in [0,1,2,3,4]:
        #     new_df[col] = np.round(new_df[col])
    for col in new_df.columns:
        new_df[col] = new_df[col] - mu[col]
        if sigma[col] > 0:
            new_df[col] = new_df[col] / sigma[col]
    return new_df


def dewhiten(data, mu, sigma, dis):
    new_df = pd.DataFrame(data, columns = list(range(data.shape[1])))
    for col in new_df.columns:
        if sigma[col] > 0:
            new_df[col] = new_df[col] * sigma[col]
        new_df[col] = new_df[col] + mu[col]
    for col in new_df.columns:
        if col in []:
            new_df[col] = np.exp(new_df[col]) - 10e-6
        if str(col) in dis or col in [1, 2, 3, 4, 6]:
            new_df[col] = np.abs(np.round(new_df[col] + 10e-4))
    return new_df

In [26]:
with open("ssl_dwa.pk", 'rb') as f:
    dwa_args = pickle.load(f)
G = generator_small(X.shape[1])
G.load_weights('ssl_gan')


In [27]:
def pred(X):
    return torch.tensor(dewhiten(G.predict(whiten(X, **dwa_args)), **dwa_args).to_numpy())

In [28]:
if classifier_config["classifier"] == "Logistic_Regression":
    classifier = LogisticRegression()
    if y.shape[1] > 1:
        # If one-hot encoding labels, pass labels as the argmax
        classifier.create_model( X.detach(), torch.argmax( y, axis=1 ) )
    else:
        classifier.create_model( X.detach(), y.ravel() ) 

In [29]:
assert ssl.shape[0] == X.shape[0]

In [33]:
ssl['pred'] = np.argmax(classifier(X).detach().numpy(), axis=1)

In [34]:
ssl['paired_pred'] = np.argmax(classifier(pred(X.numpy())).detach().numpy(), axis=1)

In [36]:
protec = ssl[ssl['RACE CODE CD'] == 'nw']

In [37]:
bad_real = protec[protec['pred'] == 0]

In [38]:
bad_both = bad_real[bad_real['paired_pred'] == 0]

In [40]:
import pickle as pk
with open("ssl_idx.pk", "wb") as f:
    pk.dump(list(bad_both.index), f)

In [3]:
ssl

,PREDICTOR RAT AGE AT LATEST ARREST,PREDICTOR RAT VICTIM SHOOTING INCIDENTS,PREDICTOR RAT VICTIM BATTERY OR ASSAULT,PREDICTOR RAT ARRESTS VIOLENT OFFENSES,PREDICTOR RAT NARCOTIC ARRESTS,PREDICTOR RAT TREND IN CRIMINAL ACTIVITY,PREDICTOR RAT UUW ARRESTS,RACE CODE CD,SSL SCORE
0,22,0,1,4,1,1.0,1,nw,0
1,28,0,9,1,0,1.0,0,w,0
2,18,1,2,1,3,1.0,0,w,0
3,17,1,1,4,0,1.6,1,nw,0
4,16,0,0,5,0,1.3,3,nw,0
...,...,...,...,...,...,...,...,...,...
398577,42,1,1,1,0,-0.1,0,nw,1
398578,32,0,0,0,0,0.3,0,nw,1
398579,70,0,0,0,1,0.1,0,nw,1
398580,40,0,0,0,1,-0.1,0,w,1


In [4]:
with open("SSL_Bayes_PREDICTOR RAT TREND IN CRIMINAL ACTIVITY_disps.pk", 'rb') as f:
    bayes_we = np.array(pickle.load(f))
with open("SSL_FACE_PREDICTOR RAT TREND IN CRIMINAL ACTIVITY_disps.pk", 'rb') as f:
    face_we = np.array(pickle.load(f))
with open("SSL_DiCE_PREDICTOR RAT TREND IN CRIMINAL ACTIVITY_disps.pk", 'rb') as f:
    DiCE_we = np.array(pickle.load(f))

with open("SSL_Bayes_PREDICTOR RAT ARRESTS VIOLENT OFFENSES_disps.pk", 'rb') as f:
    bayes_hl = np.array(pickle.load(f))
with open("SSL_FACE_PREDICTOR RAT ARRESTS VIOLENT OFFENSES_disps.pk", 'rb') as f:
    face_hl = np.array(pickle.load(f))
with open("SSL_DiCE_PREDICTOR RAT ARRESTS VIOLENT OFFENSES_disps.pk", 'rb') as f:
    DiCE_hl = np.array(pickle.load(f))

In [5]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(ncols=2, constrained_layout=False, figsize=(12,5))
ax[0].hist(bayes_we, label = 'Bayes', alpha=0.5)
ax[0].hist(face_we, label = 'FACE',alpha=0.5)
ax[0].hist(DiCE_we, label = 'DiCE',alpha=0.5)
ax[0].set_title("Criminal Activity Trend")
ax[1].hist(bayes_hl, label = 'Bayes',alpha=0.5)
ax[1].hist(face_hl, label = 'FACE',alpha=0.5)
ax[1].hist(DiCE_hl, label = 'DiCE',alpha=0.5)
ax[1].set_title("Violent Offenses")
plt.legend()
plt.savefig("german_graphs.pdf")